# Idea Pais

## Extracción de links

Politicas publicas

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de las etiquetas <article>
        articles = driver.find_elements(By.CSS_SELECTOR, "article.et_pb_post")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "a.entry-featured-image-url img")
                image_url = img.get_attribute("src")
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "h2.entry-title a")
                title = title_element.text.strip()
                link = title_element.get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "span.published").text.strip()
            except:
                date = "No especificada"

            # Contenido (puedes agregar más lógica si el contenido está en una clase específica)
            try:
                content_element = article.find_element(By.CSS_SELECTOR, "div.post-content-inner")
                content = content_element.text.strip()
            except:
                content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}{current_page}/"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

Politicas públicas

In [2]:
base_url = "https://ideapais.cl/politicas_publicas/"
ip_pp = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)

Extrayendo artículos de: https://ideapais.cl/politicas_publicas/1/


In [3]:
ip_pp

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,Reporte II Congreso de Políticas Públicas Prof...,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:29:43
1,Presentación: Labores de cuidados en la encues...,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:29:43
2,Chile desde las comunas: Propuestas locales pa...,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,2024-11-29 01:29:43
3,Radiografía: Zoom a la clase media chilena,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43
4,Radiografía: Matrícula extranjera en el sistem...,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43
5,"Apunte de Política Pública: Escenario actual, ...",https://ideapais.cl/wp-content/uploads/2024/06...,"11 de Jun, 2024",https://ideapais.cl/wp-content/uploads/2024/06...,,2024-11-29 01:29:43
6,Minuta: Natalidad y segunda transición demográ...,https://ideapais.cl/wp-content/uploads/2024/06...,"10 de Jun, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:29:43
7,Perspectivas a largo plazo: Trayectoria económ...,https://ideapais.cl/wp-content/uploads/2024/04...,"25 de Abr, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:29:43
8,Minuta: Propuestas para la Nueva Educación Púb...,https://ideapais.cl/wp-content/uploads/2024/04...,"5 de Abr, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:29:43


Que hacer con:

https://ideapais.cl/legislativo-constitucional/

Minutas legislativas

In [4]:
base_url = "https://ideapais.cl/minutas-legislativas/"
ip_ml = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
ip_ml

Extrayendo artículos de: https://ideapais.cl/minutas-legislativas/1/


,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,Minuta: Proyecto de ley que reconoce el derech...,https://ideapais.cl/wp-content/uploads/2024/07...,"Jul 15, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:30:06
1,Minuta: Proyecto de ley sobre convivencia esco...,https://ideapais.cl/wp-content/uploads/2024/06...,"Jun 25, 2024",https://ideapais.cl/wp-content/uploads/2024/06...,,2024-11-29 01:30:06
2,Minuta: Proyecto de Ley de Sala de Cuna Universal,https://ideapais.cl/wp-content/uploads/2024/04...,"Abr 5, 2024",https://ideapais.cl/wp-content/uploads/2024/04...,,2024-11-29 01:30:06
3,Minuta: Indicación sustitutiva al proyecto de ...,https://ideapais.cl/wp-content/uploads/2024/07...,"Jul 15, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:30:06
4,Minuta: Propuesta de Pacto Fiscal: Análisis y ...,https://ideapais.cl/wp-content/uploads/2024/04...,"Abr 16, 2024",https://ideapais.cl/wp-content/uploads/2024/04...,,2024-11-29 01:30:06
5,Minuta: Proyectos de ley que modifican la Ley ...,https://ideapais.cl/wp-content/uploads/2024/07...,"Jul 9, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:30:06
6,Minuta: Proyectos de ley para acompañar el emb...,https://ideapais.cl/wp-content/uploads/2024/04...,"Abr 10, 2024",https://ideapais.cl/wp-content/uploads/2024/04...,,2024-11-29 01:30:06


Discusión constitucional

In [5]:
base_url = "https://ideapais.cl/discusion-constitucional/"
ip_dc = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
ip_dc

Extrayendo artículos de: https://ideapais.cl/discusion-constitucional/1/


,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,Minuta: Propuesta de Pacto Fiscal: Análisis y ...,https://ideapais.cl/wp-content/uploads/2024/04...,"Abr 16, 2024",https://ideapais.cl/wp-content/uploads/2024/04...,,2024-11-29 01:30:23
1,Por una Constitución Democrática,https://ideapais.cl/wp-content/uploads/2022/11...,"Nov 9, 2022",https://ideapais.cl/wp-content/uploads/2022/11...,,2024-11-29 01:30:23
2,ABC Constitucional,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022",https://ideapais.cl/wp-content/uploads/2022/08...,,2024-11-29 01:30:23
3,Convención Constitucional,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23
4,Las cartas en la mesa,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23
5,Mujer y representación política,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23
6,Uso inteligente de la fuerza,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23
7,"Un camino para el tránsito constitucional, dis...",https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23
8,Policías en Chile: una mirada constitucional,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23
9,Uso inteligente de la fuerza,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 10, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:30:23


IP en los medios

In [6]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_custom_url(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL personalizada y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        time.sleep(3)  # Espera para que la página cargue completamente

        articles = driver.find_elements(By.CSS_SELECTOR, "article.wp-show-posts-single")
        data = []

        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "div.wp-show-posts-image img")
                image_url = img.get_attribute("data-src")
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "a[title]")
                title = title_element.get_attribute("title").strip()
                link = title_element.get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'URL Imagen': image_url,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages_custom(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar.
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda.
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana).
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)
    current_page = start_page
    all_articles_df = pd.DataFrame()

    while current_page <= max_pages:
        url = f"{base_url}{current_page}/"
        print(f"Extrayendo artículos de: {url}")

        all_articles_df = extract_articles_from_custom_url(driver, url, all_articles_df)

        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1

    driver.quit()
    return all_articles_df

In [7]:
base_url = "https://ideapais.cl/ip-en-los-medios/"
ip_elm = extract_articles_from_multiple_pages_custom(base_url, start_page=1, max_pages=1, headless=False)
ip_elm

Extrayendo artículos de: https://ideapais.cl/ip-en-los-medios/1/


,Título,Enlace,Fecha Publicación,URL Imagen,Fecha Consulta
0,"Columna «A dónde están, que no se ven», por Cr...",https://ideapais.cl/noticias/columna-a-donde-e...,2024-11-21T15:25:19-03:00,https://ideapais.cl/wp-content/uploads/2024/07...,2024-11-29 01:30:40
1,"Carta al Director «El parche de la confianza»,...",https://ideapais.cl/noticias/carta-al-director...,2024-11-14T16:49:17-03:00,https://ideapais.cl/wp-content/uploads/2024/11...,2024-11-29 01:30:40
2,"Columna «Los costos de mantener a Tohá», por C...",https://ideapais.cl/noticias/columna-los-costo...,2024-11-07T15:27:45-03:00,https://ideapais.cl/wp-content/uploads/2024/07...,2024-11-29 01:30:40
3,Columna «Volver a poner a las personas en el c...,https://ideapais.cl/noticias/columna-volver-a-...,2024-10-25T16:17:16-03:00,https://ideapais.cl/wp-content/uploads/2024/06...,2024-11-29 01:30:40
4,"Columna «Solidaridad UC», por Cristián Stewart...",https://ideapais.cl/noticias/columna-solidarid...,2024-10-24T10:52:13-03:00,https://ideapais.cl/wp-content/uploads/2024/07...,2024-11-29 01:30:40
5,"Columna «Los católicos y la conciencia», por C...",https://ideapais.cl/noticias/columna-los-catol...,2024-09-26T17:07:05-03:00,https://ideapais.cl/wp-content/uploads/2024/07...,2024-11-29 01:30:40
6,"Columna «CAE: víctimas y opresores», por Magda...",https://ideapais.cl/noticias/columna-cae-victi...,2024-09-18T21:47:15-03:00,https://ideapais.cl/wp-content/uploads/2024/07...,2024-11-29 01:30:40
7,"Columna «¿Es justo?», por Juan Pablo Lira | El...",https://ideapais.cl/noticias/columna-es-justo-...,2024-09-17T21:37:51-03:00,https://ideapais.cl/wp-content/uploads/2024/06...,2024-11-29 01:30:40
8,"Columna «Conmemoración de hechos importantes»,...",https://ideapais.cl/noticias/columna-conmemora...,2024-09-16T10:00:34-03:00,https://ideapais.cl/wp-content/uploads/2024/06...,2024-11-29 01:30:40


Libros y revistas

In [8]:
import time
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_products_from_custom_url(driver, url, existing_df=None):
    """
    Extrae información de productos desde una URL personalizada y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer productos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los productos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los productos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        time.sleep(3)  # Espera para que la página cargue inicialmente

        # Hacer scroll hasta el final de la página para cargar más productos
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Espera para que se carguen más productos
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Selección de elementos <li> que contienen los productos
        products = driver.find_elements(By.CSS_SELECTOR, "li.product")
        data = []

        for product in products:
            # Título del producto
            try:
                title = product.find_element(By.CSS_SELECTOR, "h2.woocommerce-loop-product__title").text.strip()
            except Exception as e:
                print(f"Error al extraer el título: {e}")
                title = None

            # Enlace al producto
            try:
                link = product.find_element(By.CSS_SELECTOR, "a.woocommerce-LoopProduct-link").get_attribute("href")
            except Exception as e:
                print(f"Error al extraer el enlace: {e}")
                link = None

            # URL de la imagen
            try:
                img = product.find_element(By.CSS_SELECTOR, "img.attachment-woocommerce_thumbnail")
                image_url = img.get_attribute("data-src") or img.get_attribute("src")
            except Exception as e:
                print(f"Error al extraer la URL de la imagen: {e}")
                image_url = None

            # Crear diccionario con los datos extraídos
            product_data = {
                'Título': title,
                'Enlace': link,
                'URL Imagen': image_url,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            data.append(product_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()


def extract_products_from_multiple_pages_custom(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los productos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar.
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda.
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana).
    
    Returns:
        pd.DataFrame: DataFrame con los productos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)
    current_page = start_page
    all_products_df = pd.DataFrame()

    while current_page <= max_pages:
        url = f"{base_url}{current_page}/"
        print(f"Extrayendo productos de: {url}")

        all_products_df = extract_products_from_custom_url(driver, url, all_products_df)

        if all_products_df is None or len(all_products_df) == 0:
            print(f"No se encontraron productos en la página {current_page}, o la página no existe.")
            break

        current_page += 1

    driver.quit()
    return all_products_df

In [9]:
base_url = "https://ideapais.cl/libros-y-revistas/"
ip_lyr = extract_products_from_multiple_pages_custom(base_url, start_page=1, max_pages=1, headless=False)
ip_lyr

Extrayendo productos de: https://ideapais.cl/libros-y-revistas/1/


,Título,Enlace,URL Imagen,Fecha Consulta
0,Revista Raíces «La familia: un bien público» V...,https://ideapais.cl/revista-raices-la-familia-...,https://ideapais.cl/wp-content/uploads/2024/04...,2024-11-29 01:31:01
1,Revista Raíces «Dilemas de la juventud» V Edición,https://ideapais.cl/revista-raices-dilemas-de-...,https://ideapais.cl/wp-content/uploads/2023/08...,2024-11-29 01:31:02
2,Revista Raíces «Pensar una centro derecha refo...,https://ideapais.cl/revista-raices-pensar-una-...,https://ideapais.cl/wp-content/uploads/2022/12...,2024-11-29 01:31:02
3,Revista Raíces «Reformismo y Constitución» III...,https://ideapais.cl/revista-raices-reformismo-...,https://ideapais.cl/wp-content/uploads/2022/08...,2024-11-29 01:31:02
4,Revista Raíces «Desafíos constitucionales» II ...,https://ideapais.cl/revista-raices-desafios-co...,https://ideapais.cl/wp-content/uploads/2022/05...,2024-11-29 01:31:02
5,Revista Raíces «Solidaridad» I Edición,https://ideapais.cl/revista-raices-solidaridad...,https://ideapais.cl/wp-content/uploads/2022/05...,2024-11-29 01:31:02
6,Gonzalo Vial: Política y Crisis Social 2da edi...,https://ideapais.cl/producto/gonzalo-vial-poli...,https://ideapais.cl/wp-content/uploads/2024/01...,2024-11-29 01:31:02
7,Cuestiones Obreras,https://ideapais.cl/producto/cuestiones-obreras/,https://ideapais.cl/wp-content/uploads/2022/05...,2024-11-29 01:31:02
8,Constitución Solidaria,https://ideapais.cl/producto/constitucion-soli...,https://ideapais.cl/wp-content/uploads/2022/05...,2024-11-29 01:31:02
9,¿Por qué ha fracasado el liberalismo? Patrick ...,https://ideapais.cl/producto/por-que-ha-fracas...,https://ideapais.cl/wp-content/uploads/2022/05...,2024-11-29 01:31:02


Noticias institucionales

In [10]:
base_url = "https://ideapais.cl/noticias-institucionales/"
ip_notins = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
ip_notins

Extrayendo artículos de: https://ideapais.cl/noticias-institucionales/1/


,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,IdeaPaís organiza conversatorio sobre desarrol...,https://ideapais.cl/noticias-institucionales/i...,"Nov 28, 2024",https://ideapais.cl/wp-content/uploads/2024/11...,,2024-11-29 01:31:18
1,Candidatos a Gobernador Regional de la región ...,https://ideapais.cl/noticias-institucionales/c...,"Oct 15, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:31:18
2,IdeaPaís invita a postular al II Congreso de P...,https://ideapais.cl/noticias-institucionales/i...,"Jun 7, 2024",https://ideapais.cl/wp-content/uploads/2024/06...,,2024-11-29 01:31:18
3,IdeaPaís inaugura sede regional en Los Lagos: ...,https://ideapais.cl/noticias-institucionales/i...,"May 30, 2024",https://ideapais.cl/wp-content/uploads/2024/05...,,2024-11-29 01:31:18
4,IdeaPaís realiza debates entre candidatos a al...,https://ideapais.cl/sin-categoria/ideapais-rea...,"Oct 17, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:31:18
5,Con presencia de Evelyn Matthei el centro de e...,https://ideapais.cl/noticias-institucionales/c...,"Sep 30, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:31:18
6,La versión 2024 del Encuentro de Líderes Jóven...,https://ideapais.cl/noticias-ohiggins/la-versi...,"Jun 6, 2024",https://ideapais.cl/wp-content/uploads/2024/06...,,2024-11-29 01:31:18
7,IdeaPaís y centros de estudios presentaron la ...,https://ideapais.cl/noticias-institucionales/i...,"May 27, 2024",https://ideapais.cl/wp-content/uploads/2024/05...,,2024-11-29 01:31:18
8,Ex autoridades del Biobío entregan sus reflexi...,https://ideapais.cl/noticias-institucionales/e...,"Oct 16, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:31:18
9,«En el Foco» de IdeaPaís analizó las eleccione...,https://ideapais.cl/noticias-institucionales/e...,"Jul 17, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:31:18


Iniciativas

In [11]:
base_url = "https://ideapais.cl/iniciativas-2/"
ip_inic = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
ip_inic

Extrayendo artículos de: https://ideapais.cl/iniciativas-2/1/


,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,Memoria 2023,https://indd.adobe.com/view/6930b94c-dbbc-4c2f...,"Nov 5, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33
1,Memoria 2020,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33
2,Memoria 2022,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33
3,Memoria 2019,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33
4,Memoria 2021,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33
5,Memoria 2018,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33


## Iteración sobre links

In [12]:
from selenium.webdriver.common.by import By

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Espera de 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1.entry-title").text.strip()
    except:
        title = "No disponible"
    
    try:
        author = driver.find_element(By.CSS_SELECTOR, "span.author.vcard a").text.strip()
    except:
        author = "No disponible"
    
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, "span.published").text.strip()
    except:
        publication_date = "No disponible"
    
    try:
        # Extraer todas las categorías (enlaces dentro de .post-meta)
        categories = [category.text.strip() for category in driver.find_elements(By.CSS_SELECTOR, ".post-meta a")]
        categories = ", ".join(categories) if categories else "No disponible"
    except:
        categories = "No disponible"
    
    try:
        content = driver.find_element(By.CSS_SELECTOR, "div.entry-content").text.strip()
    except:
        content = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Autor": author,
        "Fecha de Publicación": publication_date,
        "Categorías": categories,
        "Contenido": content,
        "Enlace": url,
    }
    
    return article_data

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [13]:
ip_elm2 = ip_elm[:2]
ip_lyr2 = ip_lyr[:2]
ip_notins2 = ip_notins[:2]

In [14]:
ip_elm_contenido = process_all_links(ip_elm)
ip_elm_contenido

Extrayendo datos de: https://ideapais.cl/noticias/columna-a-donde-estan-que-no-se-ven-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/carta-al-director-el-parche-de-la-confianza-por-arnau-sarra-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-costos-de-mantener-a-toha-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-volver-a-poner-a-las-personas-en-el-centro-por-matias-domeyko-diario-el-llanquihue/
Extrayendo datos de: https://ideapais.cl/noticias/columna-solidaridad-uc-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-catolicos-y-la-conciencia-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-cae-victimas-y-opresores-por-magdalena-vergara-diario-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-es-justo-por-juan-pablo-lira-el-dinamo/
Extrayendo datos de: https://ideapais.cl/notici

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,"Columna «A dónde están, que no se ven», por Cr...",IdeaPais,"Nov 21, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Dentro de las varias aristas que tiene el caso...,https://ideapais.cl/noticias/columna-a-donde-e...
1,"Carta al Director «El parche de la confianza»,...",María José Aravena,"Nov 14, 2024","María José Aravena, Noticias, Noticias Destaca...","Señor Director:\nEn su columna, Miriam Henríqu...",https://ideapais.cl/noticias/carta-al-director...
2,"Columna «Los costos de mantener a Tohá», por C...",IdeaPais,"Nov 7, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",A nadie le gusta que le digan cómo uno debe ha...,https://ideapais.cl/noticias/columna-los-costo...
3,Columna «Volver a poner a las personas en el c...,IdeaPais,"Oct 25, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace algunos días, en IdeaPaís organizamos un ...",https://ideapais.cl/noticias/columna-volver-a-...
4,"Columna «Solidaridad UC», por Cristián Stewart...",IdeaPais,"Oct 24, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace 14 años, nació un proyecto político en la...",https://ideapais.cl/noticias/columna-solidarid...
5,"Columna «Los católicos y la conciencia», por C...",IdeaPais,"Sep 26, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",El profesor Carlos Peña inició una importante ...,https://ideapais.cl/noticias/columna-los-catol...
6,"Columna «CAE: víctimas y opresores», por Magda...",IdeaPais,"Sep 18, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",La manera en que el Frente Amplio ha enarbolad...,https://ideapais.cl/noticias/columna-cae-victi...
7,"Columna «¿Es justo?», por Juan Pablo Lira | El...",IdeaPais,"Sep 17, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Hablemos de la asignación de recursos en educa...,https://ideapais.cl/noticias/columna-es-justo-...
8,"Columna «Conmemoración de hechos importantes»,...",IdeaPais,"Sep 16, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Sin duda que las primeras semanas de septiembr...,https://ideapais.cl/noticias/columna-conmemora...


In [15]:
ip_notins_contenido = process_all_links(ip_notins)
ip_notins_contenido

Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-organiza-conversatorio-sobre-desarrollo-urbano-con-alcaldes-de-puerto-montt-y-puerto-varas/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/candidatos-a-gobernador-regional-de-la-region-de-los-lagos-debatieron-en-el-evento-organizado-por-ideapais/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-invita-a-postular-al-ii-congreso-de-pensamiento-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-inaugura-sede-regional-en-los-lagos-es-la-cuarta-a-nivel-nacional/
Extrayendo datos de: https://ideapais.cl/sin-categoria/ideapais-realiza-debates-entre-candidatos-a-alcaldes-en-santa-cruz-y-marchigue/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/con-presencia-de-evelyn-matthei-el-centro-de-estudios-ideapais-realizo-el-congreso-de-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noti

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,IdeaPaís organiza conversatorio sobre desarrol...,María José Aravena,"Nov 28, 2024","María José Aravena, Noticias Destacadas Inicio...",Rodrigo Wainraihgt y Tomás Garate dialogaron j...,https://ideapais.cl/noticias-institucionales/i...
1,Candidatos a Gobernador Regional de la región ...,María José Aravena,"Oct 15, 2024","María José Aravena, Noticias Destacadas Inicio...",El debate se enfocó en temas de seguridad y co...,https://ideapais.cl/noticias-institucionales/c...
2,IdeaPaís invita a postular al II Congreso de P...,IdeaPais,"Jun 7, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",La instancia se realizará el 27 de septiembre ...,https://ideapais.cl/noticias-institucionales/i...
3,IdeaPaís inaugura sede regional en Los Lagos: ...,IdeaPais,"May 30, 2024","IdeaPais, Noticias IdeaPaís Los Lagos, Noticia...","En el evento, su director regional, Matías Dom...",https://ideapais.cl/noticias-institucionales/i...
4,IdeaPaís realiza debates entre candidatos a al...,María José Aravena,"Oct 17, 2024","María José Aravena, Noticias Destacadas Inicio...",El centro de estudios de inspiración social cr...,https://ideapais.cl/sin-categoria/ideapais-rea...
5,Con presencia de Evelyn Matthei el centro de e...,IdeaPais,"Sep 30, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",En el ex Congreso Nacional expusieron ex autor...,https://ideapais.cl/noticias-institucionales/c...
6,La versión 2024 del Encuentro de Líderes Jóven...,IdeaPais,"Jun 6, 2024","IdeaPais, Noticias Institucionales, Noticias O...","IdeaPaís, centro de estudios de inspiración so...",https://ideapais.cl/noticias-ohiggins/la-versi...
7,IdeaPaís y centros de estudios presentaron la ...,IdeaPais,"May 27, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",Con la presencia de la alcaldesa de Providenci...,https://ideapais.cl/noticias-institucionales/i...
8,Ex autoridades del Biobío entregan sus reflexi...,María José Aravena,"Oct 16, 2024","María José Aravena, Noticias Destacadas Inicio...",El panel de conversación se enfocó en los prin...,https://ideapais.cl/noticias-institucionales/e...
9,«En el Foco» de IdeaPaís analizó las eleccione...,IdeaPais,"Jul 17, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",IdeaPaís llevó a cabo el evento online «Europa...,https://ideapais.cl/noticias-institucionales/e...


Crear otra funcion

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    try:
        # Extraer el título
        title = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_inner h1").text.strip()
    except:
        title = "No disponible"
    
    try:
        # Extraer el año y número de forma específica (de la clase correcta)
        year_issue = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_0 .et_pb_text_inner p").text.strip()
    except:
        year_issue = "No disponible"
    
    try:
        # Extraer todo el contenido del artículo, evitando que incluya el "Año y Número"
        # Asegurarse de que el selector no esté extrayendo elementos no deseados.
        paragraphs = driver.find_elements(By.CSS_SELECTOR, ".et_pb_text_2 .et_pb_text_inner p")
        content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])  # Unir todos los párrafos
    except:
        content = "No disponible"
    
    try:
        # Esperar hasta que el enlace de descarga esté presente
        download_link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[text()='Descarga la revista aquí']"))
        )
        download_link = download_link_element.get_attribute("href")
    except Exception as e:
        print(f"Error al obtener el enlace de descarga: {e}")
        download_link = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Año y Número": year_issue,
        "Contenido": content,
        "Enlace de Descarga": download_link,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [17]:
ip_lyr_contenido = process_all_links(ip_lyr, headless=True)
ip_lyr_contenido

Extrayendo datos de: https://ideapais.cl/revista-raices-la-familia-un-bien-publico-vi-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-dilemas-de-la-juventud-v-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-pensar-una-centro-derecha-reformista-iv-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-reformismo-y-constitucion-iii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-desafios-constitucionales-ii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-solidaridad-i-edicion/
Extrayendo datos de: https://ideapais.cl/producto/gonzalo-vial-politica-y-crisis-social-2da-edicion-ampliada/
Error al obtener el enlace de descarga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A7556CB5+28821]
	(No symbol) [0x00007FF6A74C3840]
	(No symbol) [0x00007FF6A736578A]
	(No symbol) [0x00007FF6A73B91BE]
	(No symbol) [0x00007FF6A73B94AC]
	(No symbol) [0x00007FF6A7402647]
	(No symbol) [0x00007FF6A73DF33F]
	(No symbol) [0x0000

,Título,Año y Número,Contenido,Enlace de Descarga,Enlace
0,Revista Raíces «La familia: un bien público»,"AÑO 2024, Nº6",El motor del trabajo de IdeaPaís durante estos...,https://ideapais.cl/wp-content/uploads/2024/04...,https://ideapais.cl/revista-raices-la-familia-...
1,Revista Raíces «Dilemas de la juventud»,"AÑO 2023, Nº5",La quinta edición de Raíces está dedicada a re...,https://ideapais.cl/wp-content/uploads/2023/08...,https://ideapais.cl/revista-raices-dilemas-de-...
2,Revista Raíces «Pensar una centro derecha refo...,"AÑO 2022, Nº4","Este número de la revista Raíces, busca dar un...",https://ideapais.cl/wp-content/uploads/2023/02...,https://ideapais.cl/revista-raices-pensar-una-...
3,Revista Raíces «Reformismo y Constitución»,"AÑO 2022, Nº3",La tercera edición de la revista Raíces nos en...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-reformismo-...
4,Revista Raíces «Desafíos constitucionales»,"AÑO 2021, Nº2",La experiencia de los últimos años nos ha ense...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-desafios-co...
5,Revista Raíces «Solidaridad»,"AÑO 2020, Nº1","A diez años del nacimiento de IdeaPaís, presen...",https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-solidaridad...
6,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/gonzalo-vial-poli...
7,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/cuestiones-obreras/
8,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/constitucion-soli...
9,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/por-que-ha-fracas...


## Consolidar archivos

In [26]:
import pandas as pd

# Diccionario con los nombres más descriptivos para cada DataFrame
nombres_descriptivos = {
    'ip_pp': 'Políticas Públicas',
    'ip_ml': 'Minutas legislativas',
    'ip_dc': 'Discusión constitucional',
    'ip_elm_contenido': 'IP en los medios',
    'ip_lyr_contenido': 'Libros y revistas',
    'ip_notins_contenido': 'Noticias institucionales',
    'ip_inic': 'Iniciativas'
}

# Lista de DataFrames con su nombre
dataframes = [
    (ip_pp, 'ip_pp'),
    (ip_ml, 'ip_ml'),
    (ip_dc, 'ip_dc'),
    (ip_elm_contenido, 'ip_elm_contenido'),
    (ip_lyr_contenido, 'ip_lyr_contenido'),
    (ip_notins_contenido, 'ip_notins_contenido'),
    (ip_inic, 'ip_inic')
]

# Lista para almacenar los DataFrames con la columna de origen añadida
dataframes_con_origen = []

# Recorrer la lista de DataFrames, agregar la columna de origen
for df, nombre in dataframes:
    df['Categoría'] = nombre  # Agregar la columna 'origen' con el nombre del DataFrame
    dataframes_con_origen.append(df)

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes_con_origen, ignore_index=True)

# Reemplazar los nombres de la columna 'origen' con los nombres descriptivos
df_concatenado['Categoría'] = df_concatenado['Categoría'].replace(nombres_descriptivos)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta,Categoría,Autor,Fecha de Publicación,Categorías,Año y Número,Enlace de Descarga
0,Reporte II Congreso de Políticas Públicas Prof...,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:29:43,Políticas Públicas,NaN,NaN,NaN,NaN,NaN
1,Presentación: Labores de cuidados en la encues...,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:29:43,Políticas Públicas,NaN,NaN,NaN,NaN,NaN
2,Chile desde las comunas: Propuestas locales pa...,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,2024-11-29 01:29:43,Políticas Públicas,NaN,NaN,NaN,NaN,NaN
3,Radiografía: Zoom a la clase media chilena,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,Políticas Públicas,NaN,NaN,NaN,NaN,NaN
4,Radiografía: Matrícula extranjera en el sistem...,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,Políticas Públicas,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Memoria 2020,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,Iniciativas,NaN,NaN,NaN,NaN,NaN
98,Memoria 2022,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,Iniciativas,NaN,NaN,NaN,NaN,NaN
99,Memoria 2019,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,Iniciativas,NaN,NaN,NaN,NaN,NaN
100,Memoria 2021,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,Iniciativas,NaN,NaN,NaN,NaN,NaN


In [27]:
df_final = df_concatenado.copy()

In [28]:
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Idea País"

df_final  = df_final[['Título', 'Categoría', 'Enlace', 'Fecha Publicación', 'URL Imagen', 'Contenido', 'Autor', 'Fecha de Publicación', 'Categorías',
       'Año y Número', 'Enlace de Descarga']]

In [29]:
df_final

,Título,Categoría,Enlace,Fecha Publicación,URL Imagen,Contenido,Autor,Fecha de Publicación,Categorías,Año y Número,Enlace de Descarga
0,Reporte II Congreso de Políticas Públicas Prof...,Políticas Públicas,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,NaN,NaN,NaN,NaN,NaN
1,Presentación: Labores de cuidados en la encues...,Políticas Públicas,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,NaN,NaN,NaN,NaN,NaN
2,Chile desde las comunas: Propuestas locales pa...,Políticas Públicas,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,NaN,NaN,NaN,NaN,NaN
3,Radiografía: Zoom a la clase media chilena,Políticas Públicas,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,NaN,NaN,NaN,NaN,NaN
4,Radiografía: Matrícula extranjera en el sistem...,Políticas Públicas,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
97,Memoria 2020,Iniciativas,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,NaN,NaN,NaN,NaN,NaN
98,Memoria 2022,Iniciativas,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,NaN,NaN,NaN,NaN,NaN
99,Memoria 2019,Iniciativas,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,NaN,NaN,NaN,NaN,NaN
100,Memoria 2021,Iniciativas,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,NaN,NaN,NaN,NaN,NaN


## Guardar archivos

In [30]:
df_final.to_excel("Idea País.xlsx", index = False)